# Анализ изображений с помощью службы компьютерного зрения

![Робот держит фотографию](./images/computer_vision.jpg)

*Компьютерное зрение* — это отрасль искусственного интеллекта, которая исследует развитие систем ИИ, способных «видеть» мир либо в режиме реального времени через камеру, либо путем анализа изображений и видео. Это стало возможным благодаря тому, что цифровые изображения, по сути, являются просто массивами с числовыми значениями пикселей, и мы можем использовать эти значения пикселей в качестве *функций* для обучения моделей машинного обучения, которые могут классифицировать изображения, обнаруживать дискретные объекты на изображении и даже генерировать текстовые аннотации фотографий.

## Использование когнитивной службы компьютерного зрения

Microsoft Azure включает ряд *когнитивных служб*, воплощающих общие функции искусственного интеллекта, включая некоторые, которые могут помочь вам в создании решений компьютерного зрения.

Когнитивная служба *Computer Vision* служит очевидной отправной точкой нашего исследования компьютерного зрения в Azure. Она использует предварительно подготовленные модели машинного обучения для анализа изображений и извлечения информации о них.

Например, предположим, что Northwind Traders решили внедрить «умный магазин», в котором службы ИИ следят за магазином, чтобы выявлять клиентов, нуждающихся в помощи, и направлять к ним сотрудников. С помощью службы Computer Vision изображения, сделанные камерами по всему магазину, могут быть проанализированы, чтобы дать содержательное описание того, что они изображают.

### Создание ресурса Cognitive Services

Давайте начнем с создания ресурса **Cognitive Services** в вашей подписке Azure:

1. В другой вкладке браузера откройте портал Azure по адресу: https://portal.azure.com, войдя в систему под учетной записью Microsoft.
2. Нажмите кнопку **&#65291;Создать ресурс**, выполните поиск по запросу *Cognitive Services* и создайте ресурс **Cognitive Services** со следующими параметрами.
    - **Подписка**: *Ваша подписка Azure*.
    - **Группа ресурсов**: *Выберите или создайте группу ресурсов с уникальным именем.*.
    - **Регион**: *Выберите любой доступный регион*:
    - **Имя**: *Введите уникальное имя*.
    - **Ценовая категория**: классы S0
    - **Подтверждаю, что уведомление прочитано и понято**: Выбрано.
3. Дождитесь завершения развертывания. Затем перейдите на свой ресурс когнитивных служб, и на странице **Обзор** нажмите на ссылку, чтобы управлять ключами для службы. Для подключения к вашему ресурсу когнитивных служб из клиентских приложений вам понадобятся конечная точка и ключи.

### Получение ключа и конечной точки для ресурса Cognitive Services

Для использования ресурса когнитивных служб клиентским приложениям необходимы их конечная точка и ключ аутентификации:

1. На портале Azure, на странице **Ключи и конечная точка** вашего  ресурса когнитивных служб скопируйте **Ключ1** для своего ресурса и вставьте его в код ниже, заменив **YOUR_COG_KEY**.
2. Скопируйте **конечную точку** для своего ресурса и вставьте ее в код ниже, заменив **YOUR_COG_ENDPOINT**.
3. Выполните приведенный ниже код ниже, выбрав ячейку, а затем нажав кнопку **Выполнить код в ячейке** (&#9655;) слева от ячейки.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

Теперь, когда вы установили ключ и конечную точку, можно использовать службу компьютерного зрения для анализа изображения.

Выполните код в следующей ячейке, чтобы получить описание изображения в файле */data/vision/store_cam1.jpg*.

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import vision
import os
%matplotlib inline

# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam1.jpg')

# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Get a description from the computer vision service
image_stream = open(image_path, "rb")
description = computervision_client.describe_image_in_stream(image_stream)

# Display image and caption (code in helper_scripts/vision.py)
vision.show_image_caption(image_path, description)


Это кажется достаточно точным.

Давайте попробуем другое изображение.

In [ ]:
# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam2.jpg')

# Get a description from the computer vision service
image_stream = open(image_path, "rb")
description = computervision_client.describe_image_in_stream(image_stream)

# Display image and caption (code in helper_scripts/vision.py)
vision.show_image_caption(image_path, description)

Снова-таки, предложенная подпись кажется довольно точной.

## Анализ особенностей изображения

На данный момент вы использовали службу Computer Vision для создания описательных подписей для пары изображений; но вы можете сделать гораздо больше. Служба Computer Vision предоставляет возможности анализа, которые могут извлечь подробную информацию, например:

- Расположение общих типов объектов, обнаруженных на изображении.
- Местоположение и примерный возраст человеческих лиц на изображении.
- Содержит ли изображение какое-либо «взрослое», «пикантное» или «кровавое» содержимое.
- Соответствующие теги, которые могут быть связаны с изображением в базе данных, чтобы облегчить поиск.

Для анализа изображения покупателя выполните следующий код.

In [ ]:
# Get the path to an image file
image_path = os.path.join('data', 'vision', 'store_cam1.jpg')

# Specify the features we want to analyze
features = ['Description', 'Tags', 'Adult', 'Objects', 'Faces']

# Get an analysis from the computer vision service
image_stream = open(image_path, "rb")
analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=features)

# Show the results of analysis (code in helper_scripts/vision.py)
vision.show_image_analysis(image_path, analysis)

## Дополнительные сведения

В дополнение к возможностям, которые вы изучили в этой записной книжке, когнитивная служба Computer Vision включает в себя возможности:

- Распознавание на изображениях знаменитостей.
- Выявление на изображении  логотипов брендов.
- Чтение текста на изображении с помощью оптического распознавания текста (OCR).

Подробнее о когнитивной службе Computer Vision см. в [документации по Computer Vision](https://docs.microsoft.com/azure/cognitive-services/computer-vision/).
